In [1]:
import pickle
import numpy as np
from pathlib import Path

from src.input import Input_uc, Input_ed, Input_ed_prev
from src.output import Output_uc, Output_ed, Output_ed_prev
from src.unit_commitment import solve_uc
from src.economic_dispatch import solve_ed, solve_ed_prev

In [2]:
num_units=122
num_periods=24
num_buses=197

In [3]:
path_folder_processed_kpg193 = Path.cwd().resolve() / "data" / "input" / "processed" / "KPG193_ver1_2"
timestamp_2022 = np.load(Path.cwd().resolve() / "data" / "input" / "processed" / "timestamp_2022.npy")
time_start, time_end = np.datetime64("2022-07-02T00"), np.datetime64("2022-07-02T23")
idx_time_start, idx_time_end = np.where(timestamp_2022 == time_start)[0][0], np.where(timestamp_2022 == time_end)[0][0]

In [ ]:
solar_p_max = np.load(path_folder_processed_kpg193 / "solar_p_max_2022.npy")[idx_time_start-1:idx_time_end+1].sum(axis=1)
solar_p_min = np.zeros(num_periods + 1)
wind_p = np.load(path_folder_processed_kpg193 / "wind_p_2022.npy")[idx_time_start-1:idx_time_end+1].sum(axis=1)
hydro_p = np.load(path_folder_processed_kpg193 / "hydro_p_2022.npy")[idx_time_start-1:idx_time_end+1].sum(axis=1)

load = np.load(path_folder_processed_kpg193 / "demand_2022.npy")[idx_time_start-1:idx_time_end+1]
system_reserve_up = np.zeros(num_periods + 1)
system_reserve_down = np.zeros(num_periods + 1)

p_min =  np.load(path_folder_processed_kpg193 / "p_min.npy")
p_max =  np.load(path_folder_processed_kpg193 / "p_max.npy")
ramp_up =  np.load(path_folder_processed_kpg193 / "ramp_up.npy")
ramp_down =  np.load(path_folder_processed_kpg193 / "ramp_down.npy")
startup_ramp =  np.load(path_folder_processed_kpg193 / "startup_ramp.npy")
shutdown_ramp =  np.load(path_folder_processed_kpg193 / "shutdown_ramp.npy")
min_up =  np.load(path_folder_processed_kpg193 / "min_up.npy")
min_down =  np.load(path_folder_processed_kpg193 / "min_down.npy")
cost_quad =  np.load(path_folder_processed_kpg193 / "cost_quad.npy")
cost_lin =  np.load(path_folder_processed_kpg193 / "cost_lin.npy")
cost_const =  np.load(path_folder_processed_kpg193 / "cost_const.npy")

In [6]:
cost_startup_step = pickle.load(open(path_folder_processed_kpg193 / "cost_startup_step.pkl", "rb"))
num_cooling_steps = np.array([len(csc_i) for csc_i in cost_startup_step])

u_prev = [
    np.load(path_folder_processed_kpg193 / "status_2022.npy")
    [idx_time_start-num_cooling_steps.max():idx_time_start]
    [:, idx_unit][-lookup:].tolist()
    for idx_unit, lookup in enumerate(num_cooling_steps)
]

In [7]:
input_ed_prev = Input_ed_prev(
    # meta
    num_units=num_units,
    num_buses=num_buses,
    voll=3500*1300,
    let_blackout=False,
    curtail_penalty=0,
    let_curtail=False,
    exact_reserve=False,
    # renewable
    solar_p_max=solar_p_max[0],
    solar_p_min=solar_p_min[0],
    wind_p=wind_p[0],
    hydro_p=hydro_p[0],
    # system
    load=load[0],
    system_reserve_up=system_reserve_up[0],
    system_reserve_down=system_reserve_down[0],
    # u_prev
    u_prev=[u[-1] for u in u_prev],
    # operational
    p_min=p_min, p_max=p_max, cost_quad=cost_quad, cost_lin=cost_lin, cost_const=cost_const,
)

In [8]:
output_ed_prev = Output_ed_prev()
solve_ed_prev(input_ed_prev=input_ed_prev, output_ed_prev=output_ed_prev)
p_prev = output_ed_prev.p

Set parameter Username
Set parameter LicenseID to value 2659802
Academic license - for non-commercial use only - expires 2026-05-01


In [9]:
np.load(path_folder_processed_kpg193 / "mustoff_2022.npy")

array([[  45,    0, 8183],
       [  46,  864, 2111],
       [  46, 1152, 2759],
       [  47, 7296, 8759],
       [  49, 8664, 8759],
       [  51, 1785, 3287],
       [  58, 6432, 8759],
       [  60,    0, 8015],
       [  61, 1920, 6239],
       [  62,  384, 3959],
       [  63, 5736, 8759],
       [  64, 4032, 5375],
       [  65, 7944, 8759],
       [  67, 5640, 7055],
       [  68,    0, 5423],
       [  95, 6072, 6134],
       [  95, 7344, 8687],
       [  96, 6000, 7367],
       [  96, 6144, 7487],
       [  99, 1128, 3815],
       [  99, 3816, 4367],
       [ 103, 5256, 7151]])

In [10]:
p_prev

array([ 684.93627989,  602.74842662,  668.12155455,  400.        ,
        200.        ,  560.43375337,  610.06695011,  800.        ,
        800.        ,  602.34314842,  699.99999998,    0.        ,
          0.        ,  440.        ,    0.        ,  542.82106894,
        697.43212107,  682.72757022,  579.70193656,  731.09757175,
        880.        ,    0.        ,    0.        ,  850.        ,
        420.        ,  880.        ,    0.        ,    0.        ,
        899.99999209,  880.        ,  413.82712034,  209.        ,
        209.        ,  440.        ,  880.        ,    0.        ,
        595.36694109,  552.58864022,  592.42761637,  577.00906405,
        728.13945197,  586.27171678,  575.30295697,  553.66824709,
        554.21874183,    0.        , 1000.        , 1000.        ,
       1000.        , 1000.        ,  950.        ,  950.        ,
        110.        ,  865.79855456,    0.        ,  880.        ,
        500.        ,  585.60281093, 1000.        , 1000.     

---
---
---

In [11]:
solar_p_max = solar_p_max[1:]
solar_p_min = solar_p_min[1:]
wind_p = wind_p[1:]
hydro_p = hydro_p[1:]
load = load[1:]
system_reserve_up = system_reserve_up[1:]
system_reserve_down = system_reserve_down[1:]

In [12]:
load

array([[ 257.71626979,  228.51765917,  292.19618611, ...,  359.02610784,
         917.84459962,  337.41286243],
       [ 246.23522379,  218.33738705,  279.17908845, ...,  343.03179261,
         876.95538421,  322.38139935],
       [ 238.13442413,  211.15438779,  269.99448102, ...,  331.74651918,
         848.10476013,  311.77549541],
       ...,
       [ 282.20340316,  250.2304614 ,  319.95945843, ...,  393.13928274,
        1005.05439488,  369.4724362 ],
       [ 269.11870541,  238.62822727,  305.1241561 , ...,  374.91091047,
         958.45384778,  352.34140553],
       [ 259.65008514,  230.23237806,  294.38872704, ...,  361.7201178 ,
         924.73179372,  339.94469394]], shape=(24, 197))

In [13]:
input_uc = Input_uc(
    # meta
    num_units=num_units,
    num_periods=num_periods,
    num_buses=num_buses,
    voll=3500*1300,
    let_blackout=False,
    curtail_penalty=0,
    let_curtail=False,
    exact_reserve=False,
    # renewable
    solar_p_max=solar_p_max,
    solar_p_min=solar_p_min,
    wind_p=wind_p,
    hydro_p=hydro_p,
    # system
    load=load,
    system_reserve_up=system_reserve_up,
    system_reserve_down=system_reserve_down,
    # operational constraint
    p_min=p_min,
    p_max=p_max,
    ramp_up=ramp_up,
    ramp_down=ramp_down,
    startup_ramp=startup_ramp,
    shutdown_ramp=shutdown_ramp,
    min_up=min_up,
    min_down=min_down,
    # generation cost function
    cost_quad=cost_quad,
    cost_lin=cost_lin,
    cost_const=cost_const,
    # previous horizon
    p_prev=p_prev,
    u_prev=u_prev,
    # startup cost function
    cost_startup_step=cost_startup_step,
    # mustoff
    # mustoff=
)


ValueError: [Input_uc | solar_p_min] got 22, expected 24.

In [ ]:
output_uc = Output_uc()
solve_uc(input_uc=input_uc, output_uc=output_uc)

GurobiModelStatus: 3